Random Forests

Este projeto tem por objetivo desenvolver um algoritmo de Machine Learning para análise de performance de alunos em provas

Os dados foram extraídos do site do Kaggle:
https://www.kaggle.com/datasets/whenamancodes/students-performance-in-exams

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DataScience/trab3/exams-tratados1.csv', escapechar='\\', encoding='utf-8')

In [ ]:
df.head(7)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associate's degree,standard,completed,78,73,68
5,female,group D,high school,standard,none,63,77,76
6,female,group A,bachelor's degree,standard,none,62,59,63


In [ ]:
rep_df = df.replace(["associate's degree", "bachelor's degree", "master's degree"], ["associates degree", "bachelors degree", "masters degree"])
rep_df.head(7)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,male,group A,high school,standard,completed,67,67,63
1,female,group D,some high school,free/reduced,none,40,59,55
2,male,group E,some college,free/reduced,none,59,60,50
3,male,group B,high school,standard,none,77,78,68
4,male,group E,associates degree,standard,completed,78,73,68
5,female,group D,high school,standard,none,63,77,76
6,female,group A,bachelors degree,standard,none,62,59,63


In [ ]:
rep_df.shape

(1000, 8)

In [ ]:
# Transformando as variáveis categóricas nominais em variáveis categóricas ordinais
df2 = pd.DataFrame.copy(rep_df)

In [ ]:
df2['gender'].replace({'male':0, 'female': 1}, inplace=True)
df2['race/ethnicity'].replace({'group A':0, 'group B': 1, 'group C': 2, 'group D': 3, 'group E': 4}, inplace=True)
df2['lunch'].replace({'standard':0, 'free/reduced': 1}, inplace=True)
df2['test preparation course'].replace({'completed':0, 'none': 1}, inplace=True)
df2['parental level of education'].replace({'some high school':0, 'high school': 1, 'some college': 2, 
                                           'bachelors degree': 3, 'associates degree': 4, 'masters degree': 5}, inplace=True)

In [ ]:
df2.head(10)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,0,0,1,0,0,67,67,63
1,1,3,0,1,1,40,59,55
2,0,4,2,1,1,59,60,50
3,0,1,1,0,1,77,78,68
4,0,4,4,0,0,78,73,68
5,1,3,1,0,1,63,77,76
6,1,0,3,0,1,62,59,63
7,0,4,2,0,0,93,88,84
8,0,3,1,0,1,63,56,65
9,0,2,2,1,1,47,42,45


In [ ]:
df2.dtypes

gender                         int64
race/ethnicity                 int64
parental level of education    int64
lunch                          int64
test preparation course        int64
math score                     int64
reading score                  int64
writing score                  int64
dtype: object

In [ ]:
df2.shape

(1000, 8)

Legenda:

Gender: Gênero

Race/Ethnicity: Raça ou etnia

Parental Level of Education: Nível de escolaridade dos pais

Luch: Almoço (se era padrão ou de graça/reduzido)

Test Preparation Course: Curso preparatório para testes - sem é completo ou se não tem

Math Score: Pontuação em matemática

Reading Score: Pontuação em leitura

Writing Score: Pontuação em escrita

In [ ]:
# ATRIBUTOS PREVISORES E ALVO
df2.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,0,0,1,0,0,67,67,63
1,1,3,0,1,1,40,59,55
2,0,4,2,1,1,59,60,50
3,0,1,1,0,1,77,78,68
4,0,4,4,0,0,78,73,68


In [ ]:
previsores = df2.iloc[:, 0:8].values
previsores

array([[ 0,  0,  1, ..., 67, 67, 63],
       [ 1,  3,  0, ..., 40, 59, 55],
       [ 0,  4,  2, ..., 59, 60, 50],
       ...,
       [ 1,  2,  0, ..., 32, 35, 41],
       [ 1,  2,  2, ..., 73, 74, 82],
       [ 0,  0,  2, ..., 65, 60, 62]])

In [ ]:
previsores.shape

(1000, 8)

In [ ]:
alvo = df2.iloc[:, 7].values
alvo

array([ 63,  55,  50,  68,  68,  76,  63,  84,  65,  45,  85,  90,  85,
        73,  85,  57,  42,  65,  44,  31,  88,  55,  73,  54,  32,  42,
        56,  60,  89,  51,  77,  73,  39,  71,  89,  74,  75,  54,  73,
        77,  72,  64,  82,  70,  87,  51,  57,  78,  73,  57,  64,  60,
        85,  49,  47,  62,  84,  83,  51,  48,  68,  59,  62,  78,  97,
        70,  56,  49,  60,  57,  73,  47,  63,  81,  67,  70,  59,  81,
        76,  45,  62,  42,  59,  73,  39,  69,  69,  88,  61,  72,  42,
        70,  87,  93,  72,  67, 100,  49,  49,  53,  62,  79,  77,  67,
        58,  82,  73,  44,  71,  33,  86,  47,  71,  33,  74,  76,  63,
        55,  58,  87,  71,  89,  62,  45,  86,  58,  64,  45,  87, 100,
        82,  64,  57,  56,  54,  69,  76,  68,  49,  64,  57,  83,  87,
        60,  68,  57,  66,  46, 100,  44,  80,  59,  76,  73,  58,  47,
        59,  87,  93,  79,  97,  65,  91,  75,  70,  92,  58,  68,  93,
        83,  59,  76,  71,  62,  72,  60,  56,  51,  95,  86,  6

In [ ]:
alvo.shape

(1000,)

In [ ]:
# Análise das escalas dos atributos (Escalonamento)
df2.describe()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.483000,2.161000,2.144000,0.348000,0.665000,66.396000,69.002000,67.738000
std,0.499961,1.132414,1.581697,0.476574,0.472227,15.402871,14.737272,15.600985
min,0.000000,0.000000,0.000000,0.000000,0.000000,13.000000,27.000000,23.000000
25%,0.000000,1.000000,1.000000,0.000000,0.000000,56.000000,60.000000,58.000000
50%,0.000000,2.000000,2.000000,0.000000,1.000000,66.500000,70.000000,68.000000
75%,1.000000,3.000000,4.000000,1.000000,1.000000,77.000000,79.000000,79.000000
max,1.000000,4.000000,5.000000,1.000000,1.000000,100.000000,100.000000,100.000000


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [ ]:
previsores_esc

array([[-0.96655883, -1.90926764, -0.72363566, ...,  0.03923309,
        -0.13591401, -0.30385073],
       [ 1.03459817,  0.74126587, -1.35618431, ..., -1.714564  ,
        -0.67902695, -0.81689543],
       [-0.96655883,  1.62477704, -0.09108701, ..., -0.48041049,
        -0.61113783, -1.13754838],
       ...,
       [ 1.03459817, -0.1422453 , -1.35618431, ..., -2.23420758,
        -2.30836575, -1.71472367],
       [ 1.03459817, -0.1422453 , -0.09108701, ...,  0.42896578,
         0.33930981,  0.91463045],
       [-0.96655883, -1.90926764, -0.09108701, ..., -0.0906778 ,
        -0.61113783, -0.36798132]])

In [ ]:
previsoresdf = pd.DataFrame(previsores_esc)
previsoresdf

,0,1,2,3,4,5,6,7
0,-0.966559,-1.909268,-0.723636,-0.730577,-1.408927,0.039233,-0.135914,-0.303851
1,1.034598,0.741266,-1.356184,1.368782,0.709760,-1.714564,-0.679027,-0.816895
2,-0.966559,1.624777,-0.091087,1.368782,0.709760,-0.480410,-0.611138,-1.137548
3,-0.966559,-1.025756,-0.723636,-0.730577,0.709760,0.688788,0.610866,0.016802
4,-0.966559,1.624777,1.174010,-0.730577,-1.408927,0.753743,0.271421,0.016802
...,...,...,...,...,...,...,...,...
995,-0.966559,-0.142245,-0.723636,-0.730577,0.709760,0.428966,0.067753,-0.175590
996,-0.966559,0.741266,1.174010,1.368782,-1.408927,1.208431,1.493425,1.555936
997,1.034598,-0.142245,-1.356184,1.368782,0.709760,-2.234208,-2.308366,-1.714724
998,1.034598,-0.142245,-0.091087,-0.730577,0.709760,0.428966,0.339310,0.914630


RESUMO PRÉ-PROCESSAMENTO

alvo = variável que se pretende atingir.

previsores = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas manualmente, sem escalonar.

previsores_esc = conjunto de variáveis previsoras com as variáveis categóricas transformadas em numéricas, escalonada.

In [ ]:
# BASE DE TREINO E TESTE
from sklearn.model_selection import train_test_split

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(previsores_esc, alvo, test_size = 0.3, random_state = 0)

In [ ]:
x_treino.shape

(700, 8)

In [ ]:
x_teste.shape

(300, 8)

In [ ]:
y_treino.shape

(700,)

In [ ]:
y_teste.shape

(300,)

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
random = RandomForestClassifier(n_estimators=150, criterion='entropy', random_state = 0, max_depth=4)
random.fit(x_treino, y_treino)

RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=150,
                       random_state=0)

In [ ]:
previsoes_random = random.predict(x_teste)
previsoes_random

array([ 58,  78,  58,  66,  49,  48,  64,  71,  64, 100,  40,  76,  87,
        76,  71,  62,  79,  60,  86,  84,  71,  83,  70,  53,  79,  64,
        47,  64,  66,  53,  65,  60,  64,  83,  42, 100,  54,  60,  64,
        76,  84, 100,  87,  75,  71,  66,  71,  71,  64,  89,  78,  84,
        53,  70,  81,  71,  64,  71,  76,  84,  66,  64,  87,  71,  62,
        47,  82,  71,  64,  47,  71,  43,  54,  64,  71,  60,  78,  78,
        71,  58,  47,  90,  53,  70,  71,  53,  64, 100,  66,  64,  87,
        64,  62,  64,  80, 100,  42,  72,  64,  47,  56,  58,  70,  73,
        81,  56,  87,  64,  58,  75,  49,  76,  83,  84,  58,  87,  76,
        75,  71, 100, 100,  71,  62,  71,  68,  71,  71,  42,  58,  47,
        64,  70,  89,  58,  54,  80,  42,  64,  71,  58,  78,  47,  81,
        78,  47,  47,  78,  79,  54,  89,  71,  76,  64,  64,  72,  54,
        83,  64, 100,  83,  83,  78,  70,  75,  66, 100,  53,  82,  47,
        58,  56,  44,  58, 100,  47, 100,  84, 100,  66,  68,  8

In [ ]:
y_teste

array([ 58,  79,  50,  66,  45,  44,  60,  70,  63, 100,  28,  81,  87,
        75,  71,  62,  79,  60,  85,  81,  73,  83,  70,  53,  80,  60,
        51,  67,  66,  55,  65,  59,  64,  83,  43,  87,  54,  60,  67,
        77,  78,  92,  92,  75,  70,  63,  70,  83,  64,  89,  81,  86,
        50,  72,  82,  75,  65,  71,  74,  79,  66,  63,  87,  70,  61,
        48,  88,  73,  61,  52,  69,  39,  51,  61,  69,  61,  78,  89,
        76,  56,  50,  89,  50,  72,  69,  53,  64,  92,  66,  65,  88,
        60,  61,  66,  80,  96,  27,  71,  64,  49,  57,  60,  70,  73,
        81,  56,  89,  62,  59,  77,  49,  79,  82,  78,  59,  87,  76,
        76,  75,  89, 100,  71,  62,  73,  68,  74,  73,  39,  57,  50,
        60,  73,  90,  58,  54,  80,  43,  65,  71,  57,  79,  48,  82,
        80,  47,  30,  79,  83,  47,  89,  72,  77,  66,  60,  70,  53,
        82,  66,  90,  82,  82,  77,  71,  77,  65,  90,  49,  83,  47,
        59,  55,  44,  57,  99,  49,  93,  82,  91,  61,  68,  8

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
print("Acurácia: %.2f%%" % (accuracy_score(y_teste, previsoes_random) * 100.0))

Acurácia: 25.00%


In [ ]:
confusion_matrix(y_teste, previsoes_random)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 3]])

In [ ]:
print(classification_report(y_teste, previsoes_random))

              precision    recall  f1-score   support

          27       0.00      0.00      0.00         1
          28       0.00      0.00      0.00         1
          30       0.00      0.00      0.00         1
          33       0.00      0.00      0.00         1
          34       0.00      0.00      0.00         1
          35       0.00      0.00      0.00         1
          36       0.00      0.00      0.00         1
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         2
          40       0.00      0.00      0.00         1
          41       0.00      0.00      0.00         1
          42       0.00      0.00      0.00         0
          43       0.00      0.00      0.00         3
          44       1.00      0.25      0.40         4
          45       0.00      0.00      0.00         3
          47       0.10      0.67      0.17         3
          48       0.00      0.00      0.00         6
          49       0.33    

In [ ]:
# Análise dados de treino
previsoes_treino = random.predict(x_treino)
previsoes_treino

array([ 82,  60,  45,  71,  66,  70,  78,  76,  64,  56,  42,  64,  71,
        87,  76,  43,  64,  76,  47,  57,  86,  64,  76,  64,  70,  60,
        71,  72,  64,  84,  64, 100, 100,  53,  86,  64,  86,  71,  71,
        64,  60,  59,  58,  85,  87,  49,  47,  57, 100,  72,  83,  71,
        68,  83,  84,  73,  64,  42,  54,  64,  78,  65,  72,  73,  62,
        83,  71,  54,  81,  64,  67,  58,  59,  65,  58,  42,  71,  43,
        89,  76,  82,  87,  70,  71,  43,  68,  89,  64,  53,  59,  62,
        71,  64,  90,  74,  42,  79,  82,  64,  64,  86,  42,  69,  89,
        59,  71,  87,  54, 100,  83, 100,  64,  62,  67,  42,  86,  71,
       100,  64,  64,  56,  48,  78,  67,  47,  78,  41,  47,  58,  54,
        79,  58,  54,  47,  84, 100,  62,  66,  64,  53,  62, 100,  90,
        64,  62,  66,  42,  82,  47,  84,  71,  64,  83,  79,  84,  53,
        64,  74,  75,  42,  42,  59,  76,  71,  83,  64,  82,  73,  83,
       100,  89,  78,  58,  64,  53,  40,  84,  53,  60,  42,  7

In [ ]:
accuracy_score(y_treino, previsoes_treino)

0.59

In [ ]:
confusion_matrix(y_treino, previsoes_treino)

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  2],
       [ 0,  0,  0, ...,  0,  0, 15]])

Random Forest - teste e treino = 85%